池内さんの解析結果(`prdata_subj_db.csv`)に各プレスリリースの企業名情報およびプレスリリースのタイプ情報があるので、それを活用する。

```
"","year","articleid","date","etc_flg","n_subj","seqno","subj","joint","prtype"
"1","2003","NIKPRLRSP037981_06012003","20030105","0",1,1,"理経",0,"06: Order"
```

**以下は将来的な構想で、現在は未実装。現在はsubjとlisted_companyの完全一致検索を行なっている(一番下)が、それでもかなりの数が取れてきている。**

* プレスリリースとsubj, prtypeを紐つける
* prtypeでプレスリリースを分類
* listed_companyの企業名を大文字->小文字変換した後、JUMAN++で形態素に分解
* 形態素が(完全一致のための)妥当な分解になっていることを確認
* 形態素ごとの頻度を計上し、頻度の多いもの("・", "株式会社", "ホールディングス"等が出てくることが期待される)を除去
* subjを形態素に分解し、各listed companyの企業名の(高頻度単語の除去後の)形態素セットがそれぞれ完全に含まれるか判定
* 全ての形態素が完全に一対一対応した企業名をプレスリリースの企業名とする

In [1]:
data_dir = "data/"
lc_fname = "listed_company"
pr_fname = "test.csv"
#pr_fname = "pressrelease_all.csv"
pr_info_fname = "pressrelease_info.csv"

In [2]:
lc = []
with open(data_dir + lc_fname, 'r') as f:
    for line in f:
        lc.append(tuple(line.strip().split('\t')))
        
print(lc[:5])

[('アーキテクツ・スタジオ・ジャパン', '東京都港区高輪3-23-14', '6085'), ('アーク', '大阪府大阪市中央区南本町2-2-9', '7873'), ('アークコア', '東京都足立区椿2-2-2', '3384'), ('アークス', '北海道札幌市中央区南13条西11-2-32', '9948'), ('アークランドサービス', '東京都千代田区神田佐久間町1-8-4', '3085')]


In [47]:
lc_mor_remove = set(['EOS', '@', '・', 'ホールディングス', 'グループ', '　'])

In [49]:
from collections import Counter

lc_mor_all = Counter()
for comp_name, address, comp_code in lc[:30]:
    comp_name_split = run_jumanpp(comp_name)
    comp_name_mor = [x.split(' ')[0] for x in comp_name_split if x.split(' ')[0] not in lc_mor_remove]
    print(comp_name_mor)
    #for mor in comp_name_mor:
    #    lc_mor_all[mor] += 1
        
#print(sorted(lc_mor_all.items(), key = lambda x: x[1], reverse = True))

['アーキテクツ', 'スタジオ', 'ジャパン']
['アーク']
['アークコア']
['アークス']
['アーク', 'ランド', 'サービス']
['アークランドサカモト']
['アース', '製薬']
['アー', 'トス', 'パーク']
['アートネイチャー']
['アーバ', 'ネット', 'コーポレーション']
['アーバンライフ']
['アール', 'エイジ']
['アール', 'エス', 'シー']
['ＲＳ', 'Ｔｅｃｈｎｏｌｏｇｉｅｓ']
['ＲＫＢ', '毎日放送']
['アールシーコア']
['アール', 'ビバン']
['アーレスティ']
['アイ', 'アール', 'ジャパン']
['ＩＨＩ']
['アイエー']
['電通', '国際', '情報', 'サービス']
['アイ', 'エス', 'ビー']
['アイエックス', 'ナレッジ']
['ＩＭＶ']
['アイ・オー', 'データ', '機器']
['アイカ', '工業']
['愛眼']
['アイケイ']
['アイ', 'ケイ', 'ケイ']


In [38]:
lc_mor_remove = set(['EOS', '@', '・', 'ホールディングス', 'グループ', '　'])

2

In [46]:
lc_mor_all['　']

2

In [5]:
pr = []
with open(data_dir + pr_fname, 'r') as f:
    for line in f:
        pr.append(tuple(line.strip().split('\t')))
        
print(pr[:3])

[('NIKPRLRSP038042_07012003', '20030106', '発表日：２００３年１月７日マイクロドライブを４ギガバイトに拡張\u3000株式会社日立グローバルストレージテクノロジーズ（取締役社長：宮崎哲男）（＊１）は、世界最小のハードディスク装置「マイクロドライブ」の記憶容量を４ギガバイトに拡張し、２００３年秋に発売します。（＊１）\u3000株式会社日立グローバルストレージテクノロジーズはＨｉｔａｃｈｉ\u3000Ｇｌｏｂａｌ\u3000Ｓｔｏｒａｇｅ\u3000Ｔｅｃｈｎｏｌｏｇｉｅｓ（ＣＥＯ：成瀬淳、本社機能：米国カリフォルニア州）の日本法人として、２００３年１月１日に設立されました。同社はハードディスク装置の開発・製造ならびに国内における販売を担当します。\u3000\u3000\u3000マイクロドライブは、標準的なメモリーの規格であるコンパクト・フラッシュＴｙｐｅＩＩの物理的条件を満たしており、外形寸法は幅４２．８ｍｍ×奥行き３６．４ｍｍ×厚さ５．０ｍｍという小型サイズです。米国ＩＢＭ社が２０００年６月に記憶容量１ギガバイトの製品を発売して以来、大容量化を目指して開発を進めてきた結果、小型化技術を活用することで、読み書きヘッドが従来製品の半分のサイズとなる「ＦＥＭＴＯ（＊２）ヘッド」を新たに開発するとともに、ヘッドが円板上を浮上する高さ（フライト・ハイト）も約４０％下げることに成功しました。\u3000また、「ピクシー・ダスト（Ｐｉｘｉｅ\u3000Ｄｕｓｔ）」技術を利用して線記録密度を大幅に向上させています。更に、トラック密度の向上と併せて面記録密度も１００億ビット／平方センチメートルに向上させました。ピクシー・ダストとは、原子三個分相当の超薄型のプラチナに似た貴金属「ルテニウム」でできた層を、三つの磁性層で挟み込む磁気コーティング技術です。ルテニウム層と磁性層を組み合わせることによって、データの保全性を維持したまま高密度でデータを記録できるようになりました。（＊２）\u3000ＦＥＭＴＯ（フェムト）：現在使用されているピコヘッドをさらに縮小化する技術の名称です。\u3000これらの新技術によって、メディア転送速度も従来比約５０％向上し、映像メディアとして急速に普及してきたＤＶＤ（＊３）に匹敵する４ギガバイトの記憶容量を達成しま

In [3]:
pr_info = []
with open(data_dir + pr_info_fname, 'r') as f:
    for line in f:
        pr_info.append(tuple(line.strip().split('\t')))
        
print(pr_info[:3])

[('NIKPRLRSP037981_06012003', '20030105', '理経', '06: Order'), ('NIKPRLRSP037982_06012003', '20030106', 'TKC', '01: Product'), ('NIKPRLRSP037985_06012003', '20030105', 'エキスポランド', '01: Product')]


In [59]:
print([run_jumanpp(x[2]) for x in pr_info[:5]])

[['理経 りけい 理経 名詞 6 組織名 6 * 0 * 0 "自動獲得:Wikipedia Wikipedia上位語:技術商社 Wikipedia組織名"', 'EOS'], ['TKC TKC TKC 未定義語 15 アルファベット 3 * 0 * 0 "品詞推定:名詞"', 'EOS'], ['エキスポ エキスポ エキスポ 名詞 6 普通名詞 1 * 0 * 0 "自動獲得:Wikipedia Wikipedia多義"', 'ランド ランド ランド 名詞 6 普通名詞 1 * 0 * 0 "自動獲得:Wikipedia Wikipedia多義"', 'EOS'], ['ガートナー ガートナー ガートナー 名詞 6 組織名 6 * 0 * 0 "自動獲得:Wikipedia Wikipedia上位語:企業/きぎょう Wikipedia組織名"', '\u3000 \u3000 \u3000 特殊 1 空白 6 * 0 * 0 NIL', 'ジャパン じゃぱん ジャパン 名詞 6 地名 4 * 0 * 0 "代表表記:ジャパン/じゃぱん 地名:国:別称:日本"', 'EOS'], ['TKC TKC TKC 未定義語 15 アルファベット 3 * 0 * 0 "品詞推定:名詞"', 'EOS']]


In [61]:
from collections import Counter
pr_types = Counter()
for p in pr_info:
    pr_types[p[3]] += 1
#print(sorted(pr_types.items(), key = lambda x: x[1], reverse = True))
pr_types = set(pr_types.keys())
pr_types.remove('99_Others')
pr_types.remove('0')
print(pr_types)

{'04: Restructuring', '05: PR', '01: Product', '05: Price', '05: Promotion', '10: Exit', '04: License', '03: Technology', '06: Capacity', '04: Alliance', '06: Order', '04: Contract', '02: Process', '04: M&A'}


In [6]:
def run_jumanpp(s):
    import subprocess
    with open("tmp", 'w') as f:
        f.write(s)
    ret = subprocess.check_output("cat tmp | jumanpp", shell=True).decode('utf-8').strip().split('\n')
    subprocess.call("rm tmp", shell = True)
    return ret

In [62]:
# プレスリリースと企業名を結びつける
p = pr[0]
print(p)
p_info = [x[1:4] for x in pr_info if x[0] == p[0]]
print(p_info)
for comp_name in [x[0] for x in p_info]:
    compe_code = matching_comp_name(comp_name)

('NIKPRLRSP038042_07012003', '20030106', '発表日：２００３年１月７日マイクロドライブを４ギガバイトに拡張\u3000株式会社日立グローバルストレージテクノロジーズ（取締役社長：宮崎哲男）（＊１）は、世界最小のハードディスク装置「マイクロドライブ」の記憶容量を４ギガバイトに拡張し、２００３年秋に発売します。（＊１）\u3000株式会社日立グローバルストレージテクノロジーズはＨｉｔａｃｈｉ\u3000Ｇｌｏｂａｌ\u3000Ｓｔｏｒａｇｅ\u3000Ｔｅｃｈｎｏｌｏｇｉｅｓ（ＣＥＯ：成瀬淳、本社機能：米国カリフォルニア州）の日本法人として、２００３年１月１日に設立されました。同社はハードディスク装置の開発・製造ならびに国内における販売を担当します。\u3000\u3000\u3000マイクロドライブは、標準的なメモリーの規格であるコンパクト・フラッシュＴｙｐｅＩＩの物理的条件を満たしており、外形寸法は幅４２．８ｍｍ×奥行き３６．４ｍｍ×厚さ５．０ｍｍという小型サイズです。米国ＩＢＭ社が２０００年６月に記憶容量１ギガバイトの製品を発売して以来、大容量化を目指して開発を進めてきた結果、小型化技術を活用することで、読み書きヘッドが従来製品の半分のサイズとなる「ＦＥＭＴＯ（＊２）ヘッド」を新たに開発するとともに、ヘッドが円板上を浮上する高さ（フライト・ハイト）も約４０％下げることに成功しました。\u3000また、「ピクシー・ダスト（Ｐｉｘｉｅ\u3000Ｄｕｓｔ）」技術を利用して線記録密度を大幅に向上させています。更に、トラック密度の向上と併せて面記録密度も１００億ビット／平方センチメートルに向上させました。ピクシー・ダストとは、原子三個分相当の超薄型のプラチナに似た貴金属「ルテニウム」でできた層を、三つの磁性層で挟み込む磁気コーティング技術です。ルテニウム層と磁性層を組み合わせることによって、データの保全性を維持したまま高密度でデータを記録できるようになりました。（＊２）\u3000ＦＥＭＴＯ（フェムト）：現在使用されているピコヘッドをさらに縮小化する技術の名称です。\u3000これらの新技術によって、メディア転送速度も従来比約５０％向上し、映像メディアとして急速に普及してきたＤＶＤ（＊３）に匹敵する４ギガバイトの記憶容量を達成しまし

NameError: name 'matching_comp_name' is not defined

## まずは完全一致でどの程度マッチするか確かめる

In [51]:
lc_dict = {}
for comp_name, address, comp_code in lc:
    lc_dict[comp_name] = (address, comp_code)

#print(lc_dict)

{'アーキテクツ・スタジオ・ジャパン': ('東京都港区高輪3-23-14', '6085'), 'アーク': ('大阪府大阪市中央区南本町2-2-9', '7873'), 'アークコア': ('東京都足立区椿2-2-2', '3384'), 'アークス': ('北海道札幌市中央区南13条西11-2-32', '9948'), 'アークランドサービス': ('東京都千代田区神田佐久間町1-8-4', '3085'), 'アークランドサカモト': ('新潟県三条市上須頃445', '9842'), 'アース製薬': ('東京都千代田区神田司町2丁目12-1', '4985'), 'アートスパークホールディングス': ('東京都新宿区西新宿4-15-7', '3663'), 'アートネイチャー': ('東京都渋谷区代々木3-40-7', '7823'), 'アーバネットコーポレーション': ('東京都千代田区二番町5-6', '3242'), 'アーバンライフ': ('兵庫県神戸市東灘区本山南町8-6-26', '8851'), 'アールエイジ': ('東京都港区赤坂2-11-7', '3248'), 'アール・エス・シー': ('東京都豊島区東池袋3-1-3', '4664'), 'ＲＳ\u3000Ｔｅｃｈｎｏｌｏｇｉｅｓ': ('東京都品川区大井1-23-1\u3000カクタビル4F', '3445'), 'ＲＫＢ毎日放送': ('福岡県福岡市早良区百道浜2-3-8', '9407'), 'アールシーコア': ('東京都渋谷区南平台町2-17', '7837'), 'アールビバン': ('東京都品川区東品川4-13-14', '7523'), 'アーレスティ': ('愛知県豊橋市三弥町中原1‐2', '5852'), 'アイ・アールジャパンホールディングス': ('東京都千代田区霞が関3−2−5', '6035'), 'ＩＨＩ': ('東京都江東区豊洲3丁目1-1', '7013'), 'アイエーグループ': ('神奈川県横浜市戸塚区品濃町545-5', '7509'), '電通国際情報サービス': ('東京都港区港南2-17-1', '4812'), 'アイ・エス・ビー': ('東京都品川区大崎5-1-11', '9702'), 'アイエックス・ナレッジ': ('

In [53]:
count = 0
for data in pr_info:
    articleid, comp_name, pr_type = data
    if comp_name in lc_dict:
        count += 1#print(comp_name)
print(count)

93809


**住所によるマッチングは必要か？？？**

-> (こちらは同じプレスリリースに対する複数企業のヒットを分けて考えてしまっており、また、住所によるマッチングも行なっていないが、)本文検索のマッチング(82282個)と同程度もしくはそれ以上の個数をマッチングできている。今回重視したいのはむしろ正確性なので、こちらを採用した方が良いだろう

名寄せは上の方で少し試そうとしているが、今後ちゃんとできるようになったら名寄せも行いたい。

In [54]:
len(pr_info)

354691

### タイプごとにマッチング集計

-> 完全一致による検索であれば、結局必要なのはprdata_...の情報だけで、企業名が一致しないものはkabukaから株価情報を引き出す時に弾けば良い

In [64]:
print(pr_info[:3])

[('NIKPRLRSP037981_06012003', '20030105', '理経', '06: Order'), ('NIKPRLRSP037982_06012003', '20030106', 'TKC', '01: Product'), ('NIKPRLRSP037985_06012003', '20030105', 'エキスポランド', '01: Product')]


In [4]:
# articleid, prtype, date, [comp_name]
from collections import defaultdict
out_data = defaultdict(list)
for data in pr_info:
    articleid, date, comp_name, prtype = data
    out_data[(articleid, prtype, date)].append(comp_name)

In [5]:
print(list(out_data)[:3])

[('NIKPRLRSP037981_06012003', '06: Order', '20030105'), ('NIKPRLRSP037982_06012003', '01: Product', '20030106'), ('NIKPRLRSP037985_06012003', '01: Product', '20030105')]


In [7]:
with open("matchings.new", 'w') as f:
    for data, comp_name_list in out_data.items():
        f.write('\t'.join(data) + '\t' + ' '.join(comp_name_list))
        f.write("\n")